In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import re

# Helper Function
def find_tag_value(soup, tag, class_name):
    try:
        return soup.find(tag, class_=(class_name)).text.strip()
    except AttributeError:
        return None
    
def find_tag_attr(job_card, attr):
    try:
        return job_card.find('div', {'data-gtm-job-' + attr: True})['data-gtm-job-' + attr]
    except (AttributeError, KeyError):
        return None
    

list_jobs = {'golang','.net','php developer','laravel','java','python','nodejs','reactjs','nextjs','angularjs','fluter','kotlin','vuejs','backend','frontend','mobile','data analyst','data scientist','data engineer','software engineer','software developer','full-stack','programmer','javascript','user interface','user experience','hr officer','accounting officer'}

# list_jobs = {'golang'}

page = 1
data = []

for i in list_jobs:
    search_position = i
    search_position = search_position.lower().replace(' ','+')

# template = 'https://glints.com/id/opportunities/jobs/explore?'
# url_params = 'keyword={}&country=ID&locationName={}' if search_position and location else 'keyword={}&country=ID&locationName=All+Cities%2FProvinces'

    while True:
        # 1. find using input job on list
        # url = 'https://glints.com/id/opportunities/jobs/explore?keyword={}&country=ID&locationName=All+Cities%2FProvinces&page={}'.format(search_position,page)

        # 2. find using category = computer information technology
        url = 'https://glints.com/id/job-category/computer-information-technology?page={}'.format(page)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)

        # if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        job_cards = soup.find('div', class_='JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-2 bMyejJ compact_job_card')
        print(job_cards)
                
        for job_card in job_cards:
            job_title = find_tag_value(job_card, 'h3', 'CompactOpportunityCardsc__JobTitle-sc-dkg8my-9 hgMGcy')
            company_name = find_tag_value(job_card, 'a', 'CompactOpportunityCardsc__CompanyLink-sc-dkg8my-10 iTRLWx')
            job_location = find_tag_value(job_card, 'span', 'CardJobLocation__StyledTruncatedLocation-sc-1by41tq-1 kEinQH')
            work_place = find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah')
            job_tags = job_cards.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1 koGVuk')
            job_tag_values = [tag.text for tag in job_tags]
            years_experience = job_tag_values[2]
            min_education = job_tag_values[3]
            
            salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__NotDisclosedMessage-sc-dkg8my-23 hivaYx')
            if salary_range:
                salary_range = 'Not Written'
            else:
                salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__SalaryWrapper-sc-dkg8my-29 gfPeyg')
            
            skill = find_tag_attr(job_card, 'card-info').replace('experience,logo,','')
            job_id = find_tag_attr(job_card, 'id')
            work_type = find_tag_attr(job_card, 'type')
            category = find_tag_attr(job_card, 'category')
            sub_category = find_tag_attr(job_card, 'sub-category')
            role = find_tag_attr(job_card, 'role')

            more_detail_link = 'https://glints.com'+ job_card.find('a', class_= 'CompactOpportunityCardsc__CardAnchorWrapper-sc-dkg8my-24 knEIai job-search-results_job-card_link').get('href')
            # detail link request
            response_detail = requests.get(more_detail_link, headers=headers)
            soup_detail = BeautifulSoup(response_detail.content, 'html.parser')
            job_description = find_tag_value(soup_detail,'div','JobDescriptionsc__DescriptionContainer-sc-22zrgx-2 jCwTA-d')

            print(f"Job Title: {job_title}")
            print(f"Company Name: {company_name}")
            print(f"Experience: {years_experience}")
            print(f"Education: {min_education}")
            print(f"Work Place: {work_place}")
            print(f"Work Type: {work_type}")
            # print(f"Posted Date: {posted_date}")
            print(f"Salary: {salary_range}")
            print(f"Detail Description: {job_description}")
            
            data.append({
                'job_title': job_title,
                'company_name': company_name,
                'job_location': job_location,
                'work_place': work_place,
                'salary_range': salary_range,
                'more_detail_link': more_detail_link,
                'job_id': job_id,
                'work_type': work_type,
                'workplace': work_place,
                'years_experience': years_experience,
                'min_education': min_education,
                'category': category,
                'sub_category': sub_category,
                'role': role
            })
            
        page += 1
        next_button = soup.find('a', class_='pagination__link pagination__next')
        if not next_button:
            print('No more pages to fetch')
            break
        
        else:
            print(f"Failed to fetch URL: {url}")
            break
    break



# df = pd.DataFrame(data)

# script_dir = os.path.dirname(os.path.abspath(__file__))
# custom_name = f'list_of_jobs_glints_1.csv'
# file_path = os.path.join(script_dir, custom_name)
# df.to_csv(file_path, index=False, encoding='utf-8')

# # connect to mongodb

# import pymongo
# from pymongo import MongoClient

# # connect to mongodb
 
# client = MongoClient('mongodb://localhost:27017/')
# db = client['jobs']
# collection = db['glints']

# # convert dataframe to dictionary
# data_dict = df.to_dict("records")

# # insert data to mongodb
# collection.insert_many(data_dict)

# # check data in mongodb
# cursor = collection.find({})
# for document in cursor:
#     print(document)






    

<div class="JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-2 bMyejJ compact_job_card"><div aria-label="Job Card" class="JobCardsc__JobCardWrapper-sc-hmqj50-1 fnsbDp" data-position="0" role="presentation" tabindex="0"><a class="CompactOpportunityCardsc__CardAnchorWrapper-sc-dkg8my-24 knEIai job-search-results_job-card_link" href="/id/opportunities/jobs/back-end-developer/fbcc6efd-3e92-47c8-848b-28fd57ee1478?utm_referrer=explore" target="_blank"></a><div aria-label="Job card 0" class="CompactOpportunityCardsc__CompactJobCard-sc-dkg8my-3 dmlEjD" data-gtm-is-confidential-company="false" data-gtm-is-hot-job="true" data-gtm-job-card-info="experience,salary,logo,Laravel,PHP,Python,Go / Golang" data-gtm-job-category="Computer &amp; Information Technology" data-gtm-job-company-id="5280573f-4de2-491e-b4a5-b35c0a388615" data-gtm-job-id="fbcc6efd-3e92-47c8-848b-28fd57ee1478" data-gtm-job-idx="0" data-gtm-job-role="Backend Developer" data-gt

In [ ]:
import csv
import sys
import pandas as pd
import re



# Merging 2 CSV Files

In [ ]:
df = pd.read_csv('list_of_jobs_glints.csv')
df2 = pd.read_csv('list_of_jobs_glints_2.csv')

In [ ]:

merged_df = pd.concat([df, df2])
merged_df.to_csv('merged_file.csv', index=False)


# Cleaning Merged File 

In [ ]:
# remove duplicate based on job_id
merged_df = merged_df.drop_duplicates(subset='job_id', keep='first')

In [ ]:
merged_df.to_csv('merged_file.csv', index=False)

In [ ]:
merged_file = pd.read_csv('merged_file.csv')

## Cleaning Company Name Column

In [ ]:
def clean_column(job_dataset, column_name):
    corpus = []
    for i in range(0, len(job_dataset)):
        text = re.sub(r'[."]', '', job_dataset[column_name][i])
        text = text.upper()
        text = text.split()
        text = ' '.join(text)
        corpus.append(text)

    job_dataset[column_name] = corpus
    return job_dataset

In [ ]:
clean_column(merged_file,'company_name')

In [ ]:
merged_file['role'].value_counts().to_csv('role.csv')

In [ ]:
merged_file['sub_category'].value_counts().to_csv('sub_cat.csv')